In [23]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import os
import datetime as datetime
import pickle as pkl

In [24]:
filtered_train = pd.read_csv('/home/deepin/Desktop/projects/hrv/WESAD/WESAD/S3chest.csv')

In [25]:
filtered_train.columns

Index(['ID', 'chestACCx', 'chestACCy', 'chestACCz', 'chestECG', 'chestEMG',
       'chestEDA', 'chestTemp', 'chestResp', 'label'],
      dtype='object')

In [26]:
filtered_train.shape

(4545100, 10)

In [27]:
df=filtered_train["label"]
df

0          0
1          0
2          0
3          0
4          0
          ..
4545095    0
4545096    0
4545097    0
4545098    0
4545099    0
Name: label, Length: 4545100, dtype: int64

In [28]:
a=np.unique(filtered_train['label'])
a

array([0, 1, 2, 3, 4, 5, 6, 7])

In [29]:
filtered_train = filtered_train.drop(['ID'], axis=1)

In [30]:
filtered_train = filtered_train[filtered_train['label']<4]
filtered_train = filtered_train[filtered_train['label']!=0]
#print(filtered_train['ID'].unique())
print(filtered_train['label'].unique())

[1 2 3]


In [31]:
filtered_train.corr()['label']

chestACCx    0.465554
chestACCy    0.299032
chestACCz    0.845686
chestECG    -0.000182
chestEMG    -0.007106
chestEDA     0.520011
chestTemp    0.297114
chestResp   -0.000205
label        1.000000
Name: label, dtype: float64

In [32]:
filtered_target = filtered_train['label']
filtered_train = filtered_train.drop(['label','chestECG',"chestResp"], axis=1)

In [33]:
print(filtered_target.shape)
print(filtered_train.shape)
filtered_target=filtered_target.replace({1:0,2:1,3:2})
filtered_train=filtered_train[0:1508500]
filtered_target=filtered_target[0:1508500]

(1508500,)
(1508500, 6)


In [34]:
print(filtered_target.shape)
print(filtered_train.shape)

(1508500,)
(1508500, 6)


In [35]:
filtered_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1508500 entries, 246998 to 3765197
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   chestACCx  1508500 non-null  float64
 1   chestACCy  1508500 non-null  float64
 2   chestACCz  1508500 non-null  float64
 3   chestEMG   1508500 non-null  float64
 4   chestEDA   1508500 non-null  float64
 5   chestTemp  1508500 non-null  float64
dtypes: float64(6)
memory usage: 80.6 MB


In [36]:
filtered_target.info()

<class 'pandas.core.series.Series'>
Int64Index: 1508500 entries, 246998 to 3765197
Series name: label
Non-Null Count    Dtype
--------------    -----
1508500 non-null  int64
dtypes: int64(1)
memory usage: 23.0 MB


In [37]:
a=np.unique(filtered_target)
a

array([0, 1, 2])

In [38]:
filtered_train.describe()

,chestACCx,chestACCy,chestACCz,chestEMG,chestEDA,chestTemp
count,1.508500e+06,1.508500e+06,1.508500e+06,1.508500e+06,1.508500e+06,1.508500e+06
mean,8.715897e-01,-8.104693e-03,-1.941080e-01,-3.438148e-03,7.563662e+00,3.202233e+01
std,6.660401e-02,6.834795e-02,2.155274e-01,2.607120e-02,9.664791e-01,2.443908e+00
min,-6.600000e+00,-6.600000e+00,-6.600000e+00,-1.500000e+00,0.000000e+00,-2.731500e+02
25%,8.386000e-01,-4.619998e-02,-3.926000e-01,-1.240540e-02,6.628418e+00,3.132520e+01
50%,8.570000e-01,-1.580000e-02,-3.336000e-01,-3.433228e-03,7.557297e+00,3.181219e+01
75%,9.090000e-01,2.680004e-02,1.499999e-02,5.401611e-03,8.146667e+00,3.311304e+01
max,1.100600e+00,5.125999e-01,8.714000e-01,1.047180e+00,1.209869e+01,3.452197e+01


In [39]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
# shuffle X and y
X_shuffled, y_shuffled = shuffle(filtered_train, filtered_target, random_state=42)

# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_shuffled, y_shuffled, test_size=0.2, random_state=42)


In [40]:
X_train.shape

(1206800, 6)

In [41]:
from keras.models import Sequential 
from keras.layers import Dense,Activation,Dropout 
from keras.utils import np_utils
from tensorflow.keras.layers import BatchNormalization
from keras.optimizers import Adam



In [42]:
from keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=3)
y_test = to_categorical(y_test, num_classes=3)


# first neural network

# second neural network

In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
# Define the model

model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# Compile the model
optimizer = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Define early stopping and learning rate reduction callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, min_lr=0.00001)

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=500, validation_data=(X_test, y_test), callbacks=[early_stopping, reduce_lr])
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

/home/deepin/.local/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/100
2414/2414 [==============================] - 29s 11ms/step - loss: 0.1718 - accuracy: 0.9324 - val_loss: 1.5502 - val_accuracy: 0.6395 - lr: 0.0010
Epoch 2/100
2414/2414 [==============================] - 26s 11ms/step - loss: 0.0632 - accuracy: 0.9793 - val_loss: 2.9303 - val_accuracy: 0.7505 - lr: 0.0010
Epoch 3/100
2414/2414 [==============================] - 28s 11ms/step - loss: 0.0525 - accuracy: 0.9833 - val_loss: 2.8688 - val_accuracy: 0.8149 - lr: 0.0010
Epoch 4/100
2414/2414 [==============================] - 27s 11ms/step - loss: 0.0449 - accuracy: 0.9860 - val_loss: 0.0416 - val_accuracy: 0.9951 - lr: 0.0010
Epoch 5/100
2414/2414 [==============================] - 27s 11ms/step - loss: 0.0406 - accuracy: 0.9873 - val_loss: 2.5673 - val_accuracy: 0.8235 - lr: 0.0010
Epoch 6/100
2414/2414 [==============================] - 27s 11ms/step - loss: 0.0369 - accuracy: 0.9886 - val_loss: 2.3867 - val_accuracy: 0.8247 - lr: 0.0010
Epoch 7/100
2414/2414 [=================

In [44]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                448       
                                                                 
 batch_normalization_4 (Batc  (None, 64)               256       
 hNormalization)                                                 
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 batch_normalization_5 (Batc  (None, 32)               128       
 hNormalization)                                                 
                                                                 
 dropout_5 (Dropout)         (None, 32)               

In [47]:
model.save('model.h5')

In [ ]:
import pickle

# assume you have a Keras model named `model`
# ...

# save the model to a pickle file
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)


In [ ]:
import tensorflow as tf
from tensorflow import keras
model = tf.keras.models.load_model('model.h5')
